In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\Depression_ML_poly.xlsx')
df.head()

,Sample_ID,Residence,Hukou,Estates,Debt,Income,Gender,Age,Education,Marriage,...,ZStress ZEBS_SF,ZStress ZChronic,ZStress ZMedCompliance,ZWAFCS ZEBS_SF,ZWAFCS ZChronic,ZWAFCS ZMedCompliance,ZEBS_SF ZChronic,ZEBS_SF ZMedCompliance,ZChronic ZMedCompliance,Depression
0,1,2,2,2,1,7,2,1,6,0,...,0.205568,0.170308,0.289780,0.347607,0.287984,0.490005,0.080570,0.137089,0.113575,0
1,2,2,2,1,0,0,2,1,6,0,...,0.043801,-0.106094,-0.057838,-0.371081,0.898828,0.490005,-0.268449,-0.146347,0.354480,0
2,3,2,2,3,1,1,2,4,3,1,...,1.081044,0.575532,0.405650,0.445963,0.237424,0.167343,0.730677,0.515000,0.274179,0
3,4,1,1,1,0,2,2,0,2,0,...,-0.570451,-1.435338,0.753268,-0.371081,-0.933695,0.490005,0.278862,-0.146347,-0.368231,1
4,5,2,2,0,0,0,1,1,6,0,...,0.307050,1.446502,-0.405452,-0.371081,-1.748155,0.490005,0.522113,-0.146347,-0.689438,1


In [3]:
X = df.drop('Depression', axis = 1)
X.shape

(11031, 562)

In [4]:
y=df['Depression']
y.shape

(11031,)

In [5]:
pd.Series(y).value_counts()

Depression
0    8832
1    2199
Name: count, dtype: int64

In [6]:
smote = SMOTE()

In [8]:
X.dtypes

Sample_ID                    int64
Residence                    int64
Hukou                        int64
Estates                      int64
Debt                         int64
                            ...   
ZWAFCS ZChronic            float64
ZWAFCS ZMedCompliance      float64
ZEBS_SF ZChronic           float64
ZEBS_SF ZMedCompliance     float64
ZChronic ZMedCompliance    float64
Length: 562, dtype: object

In [9]:
X = X.apply(pd.to_numeric, errors='coerce')

In [10]:
X.dtypes

Sample_ID                    int64
Residence                    int64
Hukou                        int64
Estates                      int64
Debt                         int64
                            ...   
ZWAFCS ZChronic            float64
ZWAFCS ZMedCompliance      float64
ZEBS_SF ZChronic           float64
ZEBS_SF ZMedCompliance     float64
ZChronic ZMedCompliance    float64
Length: 562, dtype: object

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [12]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
print(y_train.value_counts())  # 原始训练集的类别分布
print(y_train_resampled.value_counts())  # 过采样后的类别分布

Depression
0    7093
1    1731
Name: count, dtype: int64
Depression
1    7093
0    7093
Name: count, dtype: int64


In [14]:
df2=pd.concat([y_train_resampled, X_train_resampled], axis=1)

In [15]:
df2.to_excel(r'C:\Users\Lenovo\Desktop\ML_health\Depression_ML_poly_smote.xlsx',index = False)

In [16]:
df3 = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\Depression_ML_poly_smote.xlsx')

In [17]:
X = df3.drop('Depression', axis = 1)
X.shape

(14186, 562)

In [18]:
y=df3['Depression']
y.shape

(14186,)

In [19]:
X = df3.drop(['Depression', 'Sample_ID'], axis=1)

In [19]:
X.shape

(14186, 561)

In [20]:
y=df3['Depression']
y.shape

(14186,)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [22]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [23]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.79      0.67      0.73      1457
           1       0.70      0.81      0.76      1381

    accuracy                           0.74      2838
   macro avg       0.75      0.74      0.74      2838
weighted avg       0.75      0.74      0.74      2838

[[ 983  474]
 [ 256 1125]]
Accuracy: 0.7427766032417196
Precision: 0.7484732437192131
Recall: 0.7446505347353061
F1 Score: 0.742131021846932
ROC AUC: 0.8138907926328339
Specificity: 0.6746739876458476


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [26]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1457
           1       0.81      0.83      0.82      1381

    accuracy                           0.83      2838
   macro avg       0.83      0.83      0.83      2838
weighted avg       0.83      0.83      0.83      2838

[[1193  264]
 [ 232 1149]]
Accuracy: 0.8252290345313601
Precision: 0.8251782322047156
Recall: 0.825405779087399
F1 Score: 0.8251870147043556
ROC AUC: 0.8254057790873991
Specificity: 0.818805765271105


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [30]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [31]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.93      0.74      0.83      1457
           1       0.78      0.94      0.85      1381

    accuracy                           0.84      2838
   macro avg       0.85      0.84      0.84      2838
weighted avg       0.85      0.84      0.84      2838

[[1083  374]
 [  83 1298]]
Accuracy: 0.8389711064129669
Precision: 0.8525661280129999
Recall: 0.8416033958263859
F1 Score: 0.8380415930581584
ROC AUC: 0.9247518906703736
Specificity: 0.7433081674673988


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [34]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=2000)

In [35]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [36]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      1457
           1       0.89      0.88      0.89      1381

    accuracy                           0.89      2838
   macro avg       0.89      0.89      0.89      2838
weighted avg       0.89      0.89      0.89      2838

[[1309  148]
 [ 162 1219]]
Accuracy: 0.8907681465821
Precision: 0.8908022798239756
Recall: 0.8905575570406692
F1 Score: 0.8906581835970124
ROC AUC: 0.955747603146338
Specificity: 0.8984214138641043


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [38]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [39]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 41, 51, 61]
}

# 创建 LightGBM 分类器
gbm = LGBMClassifier()

# 创建网格搜索对象
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 最佳参数和最佳模型
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4570, number of negative: 4508
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 110321
[LightGBM] [Info] Number of data points in the train set: 9078, number of used features: 561
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503415 -> initscore=0.013660
[LightGBM] [Info] Start training from score 0.013660
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4569, number of negative: 4509
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.051643 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 110316
[LightGBM] [Info] N

In [51]:
y_pred = best_model.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      1457
           1       0.94      0.92      0.93      1381

    accuracy                           0.93      2838
   macro avg       0.93      0.93      0.93      2838
weighted avg       0.93      0.93      0.93      2838

[[1378   79]
 [ 112 1269]]
Accuracy: 0.9326990838618746
Precision: 0.933113436759405
Recall: 0.9323391731196546
F1 Score: 0.9325996600750301
ROC AUC: 0.9822843303843664
Specificity: 0.9457789979409746


In [53]:
# 获取特征重要性
feature_importances = best_model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

                 Feature  Importance
28               ZStress         182
118    Estates ZEQ_5D_5L         179
544  ZEQ5D5L_VS ZChronic         174
549   ZHLS_SF12 ZChronic         173
531     ZFHS_SF ZChronic         170
..                   ...         ...
106      Estates Smoking           0
133      Debt Employment           0
332   Employment Alcohol           0
135         Debt Smoking           0
190       Gender Smoking           0

[561 rows x 2 columns]
                       Feature  Importance
28                     ZStress         182
118          Estates ZEQ_5D_5L         179
544        ZEQ5D5L_VS ZChronic         174
549         ZHLS_SF12 ZChronic         173
531           ZFHS_SF ZChronic         170
558           ZEBS_SF ZChronic         155
541         ZEQ5D5L_VS ZStress         155
488  ZAgreeableness ZEQ5D5L_VS         153
556            ZWAFCS ZChronic         151
429          ZBMI ZNeuroticism         141


In [54]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [55]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [56]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [57]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1457
           1       0.90      0.93      0.91      1381

    accuracy                           0.91      2838
   macro avg       0.91      0.91      0.91      2838
weighted avg       0.91      0.91      0.91      2838

[[1307  150]
 [  95 1286]]
Accuracy: 0.9136715997181113
Precision: 0.9138914165597098
Recall: 0.914128999456791
F1 Score: 0.9136668726563091
ROC AUC: 0.9734088524673268
Specificity: 0.8970487302676733


In [58]:
# 获取特征重要性
feature_importances = model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

                Feature  Importance
91        Hukou ZStress    0.019056
121     Estates ZStress    0.017218
256   Education ZStress    0.016602
205      Gender ZStress    0.016246
60    Residence ZStress    0.015688
..                  ...         ...
265    Marriage Smoking    0.000134
351  Occupation Smoking    0.000131
106     Estates Smoking    0.000124
45    Residence Smoking    0.000121
135        Debt Smoking    0.000096

[561 rows x 2 columns]
               Feature  Importance
91       Hukou ZStress    0.019056
121    Estates ZStress    0.017218
256  Education ZStress    0.016602
205     Gender ZStress    0.016246
60   Residence ZStress    0.015688
24             ZFHS_SF    0.015479
28             ZStress    0.015387
207     Gender ZEBS_SF    0.015072
93       Hukou ZEBS_SF    0.013459
62   Residence ZEBS_SF    0.012546


In [59]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train, y_train)
# 定义参数网格
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 设置网格搜索
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best parameters:", grid_search.best_params_)

# 获取最佳模型
best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


D:\anaconda\envs\MentalAssessment\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [61]:
y_pred = best_rf.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [62]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_rf.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      1457
           1       0.90      0.93      0.92      1381

    accuracy                           0.92      2838
   macro avg       0.92      0.92      0.92      2838
weighted avg       0.92      0.92      0.92      2838

[[1316  141]
 [  91 1290]]
Accuracy: 0.9182522903453136
Precision: 0.9183954441628336
Recall: 0.9186657634720048
F1 Score: 0.9182454286100272
ROC AUC: 0.9751125307325569
Specificity: 0.9032258064516129


In [63]:
# 获取特征重要性
feature_importances = best_rf.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

               Feature  Importance
28             ZStress    0.021071
121    Estates ZStress    0.016633
91       Hukou ZStress    0.016510
205     Gender ZStress    0.014942
60   Residence ZStress    0.014937
..                 ...         ...
45   Residence Smoking    0.000158
372  Smoking P_Alcohol    0.000136
371    Smoking Alcohol    0.000135
265   Marriage Smoking    0.000127
135       Debt Smoking    0.000120

[561 rows x 2 columns]
               Feature  Importance
28             ZStress    0.021071
121    Estates ZStress    0.016633
91       Hukou ZStress    0.016510
205     Gender ZStress    0.014942
60   Residence ZStress    0.014937
30             ZEBS_SF    0.013871
178     Income ZStress    0.012608
231        Age ZStress    0.012572
24             ZFHS_SF    0.012344
256  Education ZStress    0.012294
